In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import *

## 6. Scheduling

***Example***<br>
A post office requires different numbers of full-time employees on different days of the week. The number of full-time employees required on each day is given below. Union rules state that each full-time employee must work five consecutive days and then receive two days off. For example, an employee who works Monday to Friday must be off on Saturday and Sunday. The post office wants to meet its daily requirements using only full-time employees. Formulate an LP that the post office can use to minimize the number of full-time employees who must be hired.

| Day | Number of Full-time Employees Required |
|-----|----------------------------------------|
| 1 = Monday | 17 |
| 2 = Tuesday | 13 |
| 3 = Wednesday | 15 |
| 4 = Thursday | 19 |
| 5 = Friday | 14 |
| 6 = Saturday | 16 |
| 7 = Sunday | 11 |


***Solution***

\begin{align*}
\text{total number of employees } \quad & E = x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7\\
\quad\\
\min \quad & E\\
\text{s.t.} \quad   &   \text{Monday} = x_1 + x_4 + x_5 + x_6 + x_7 &\ge 17\\
            \quad   &   \text{Tuesday} = x_1 + x_2 + x_5 + x_6 + x_7 &\ge 13\\
            \quad   &   \text{Wednesday} = x_1 + x_2 + x_3 + x_6 + x_7 &\ge 15\\
            \quad   &   \text{Thursday} = x_1 + x_2 + x_3 + x_4 + x_7 &\ge 19\\
            \quad   &   \text{Friday} = x_1 + x_2 + x_3 + x_4 + x_5 &\ge 14\\
            \quad   &   \text{Saturday} = x_2 + x_3 + x_4 + x_5 + x_6 &\ge 16\\
            \quad   &   \text{Sunday} = x_3 + x_4 + x_5 + x_6 + x_7 &\ge 11\\
            \quad   &   x_i &\in \mathbb{Z}^+ && \forall i \in \{1,2,3,4,5,6,7\}
\end{align*}

In [8]:
days = range(7)
emps = [17, 13, 15, 19, 14, 16, 11]
n_days = len(days)
day_dict = dict(zip(days, ["Mon", "Tues", "Wed", "Thurs", "Fri", "Sat", "Sun"]))

model = LpProblem("min_employees", LpMinimize)

x = LpVariable.dicts("x", days, lowBound=0, cat="Integer")

model += lpSum([x[i] for i in days])

for i in days:
    model += lpSum(([x[i] + x[(i-4)%n_days]] + x[(i-3)%n_days] + x[(i-2)%n_days] + x[(i-1)%n_days])) >= emps[i]

model.solve()

if model.status == 1:
    for k, v in x.items():
        print(f"{day_dict[k]} \t= {v.varValue:,.0f} employees")
    print()
    print(f"Minimum number of employees to hire: {model.objective.value():,.0f}")

Mon 	= 2 employees
Tues 	= 6 employees
Wed 	= 0 employees
Thurs 	= 7 employees
Fri 	= 0 employees
Sat 	= 3 employees
Sun 	= 5 employees

Minimum number of employees to hire: 23


### Problems

1. In the post office example, suppose that each full-time employee works 8 hours per day. Thus, Monday’s requirement of 17 workers may be viewed as a requirement of 8(17) = 136 hours. The post office may meet its daily labor requirements by using both full-time and part-time employees. During each week, a full-time employee works 8 hours a day for five consecutive days, and a part-time employee works 4 hours a day for five consecutive days. A full-time employee costs the post office $15 per hour, whereas a part-time employee (with reduced fringe benefits) costs the post office only $10 per hour. Union requirements limit part-time labor to 25% of weekly labor requirements. Formulate an LP to minimize the post office’s weekly labor costs.

| Day | Hours Required
|-----|----------------------------------------
| 0 = Monday | 136
| 1 = Tuesday | 104 
| 2 = Wednesday | 120 
| 3 = Thursday | 152 
| 4 = Friday | 112 
| 5 = Saturday | 128 
| 6 = Sunday | 88 


\begin{aligned}
\min \quad & \sum^6_{i=0}15f_i + 10p_i
\quad\\
\text{s.t.} \quad   &   8f_i + 4p_i & \ge \text{hours required for day } i && \forall i \in \{0,...,6\}\\
            \quad   &   \sum^{6}_{i=0}4p_i &\le \sum^{6}_{i=0}0.25(8f_i)\\
            \quad   &   f_i, p_i    &\in    \mathbb{Z}^+  && \forall i \in \{0,...,6\}
\end{aligned}

In [9]:
hours = [136, 104, 120, 152, 112, 128, 88]
f_c = 15
p_c = 10
f_h = 8
p_h = 4

hours = dict(zip(days, hours))

f = LpVariable.dicts("f", days, lowBound=0, cat="Integer")
p = LpVariable.dicts("p", days, lowBound=0, cat="Integer")

model = LpProblem("minimize_cost", LpMinimize)

model += f_c * lpSum(f[i] for i in days)  + p_c * lpSum(p[i] for i in days)

for i in days:
    model += f_h * f[i] + p_h * p[i] >= hours[i]

model += lpSum([4 * p[i] for i in days]) <= 0.25 * lpSum([8 * f[i] for i in days])

model.solve()

if model.status == 1:
    for (k0, v0), (k1, v1) in zip(f.items(), p.items()):
        print(f"f{k0} = {v0.varValue} | p{k1} = {v1.varValue}")

f0 = 17.0 | p0 = 0.0
f1 = 13.0 | p1 = 0.0
f2 = 15.0 | p2 = 0.0
f3 = 19.0 | p3 = 0.0
f4 = 14.0 | p4 = 0.0
f5 = 16.0 | p5 = 0.0
f6 = 11.0 | p6 = 0.0


2. During each 4-hour period, the Smalltown police force requires the following number of on-duty police officers: 12 midnight to 4 A.M.—8; 4 to 8 A.M.—7; 8 A.M. to 12 noon—6; 12 noon to 4 P.M.—6; 4 to 8 P.M.—5; 8 P.M. to 12 midnight—4. Each police officer works two consecutive 4-hour shifts. Formulate an LP that can be used to minimize the number of police officers needed to meet Smalltown’s daily requirements.

Time| Idx| Officers
-|-|-
12AM - 4AM|0|8
4AM - 8AM|1|7
8AM - 12PM|2|6
12PM - 4PM|3|6
4PM - 8PM|4|5
8PM - 12AM|5|4

\begin{aligned}
\min    \quad   &   \sum^5_{i=0}o_i\\
\text{s.t.}    \quad   &   o_i + o_{i-1} &\ge \text{officers needed in chunk } i\\
        \quad   &   o_i &\in    \mathbb{Z}^+
\end{aligned}

In [10]:
chunks = range(6)
officers = [8, 7, 6, 6, 5, 4]
n_chunks = len(chunks)
time_dict = dict(zip(chunks, ["12AM - 4AM", "4AM - 8AM", "8AM - 12PM", "12PM - 4PM", "4PM - 8PM", "8PM - 12AM"]))

model = LpProblem("min_OFFICERS", LpMinimize)

o = LpVariable.dicts("o", chunks, lowBound=0, cat="Integer")

model += lpSum([o[i] for i in chunks])

for i in chunks:
    model += lpSum([o[i] + o[(i-1)%n_chunks]]) >= officers[i]

model.solve()

if model.status == 1:
    for k, v in o.items():
        print(f"{time_dict[k]} \t= {v.varValue:,.0f} officers")
    print()
    print(f"Minimum number of officers to hire: {model.objective.value():,.0f}")

12AM - 4AM 	= 2 officers
4AM - 8AM 	= 5 officers
8AM - 12PM 	= 1 officers
12PM - 4PM 	= 5 officers
4PM - 8PM 	= 0 officers
8PM - 12AM 	= 6 officers

Minimum number of officers to hire: 19


3. During each 6-hour period of the day, the Bloomington Police Department needs at least the number of policemen shown below. Policemen can be hired to work either 12 consecutive hours or 18 consecutive hours. Policemen are paid $4 per hour for each of the first 12 hours a day they work and are paid $6 per hour for each of the next 6 hours they work in a day. Formulate an LP that can be used to minimize the cost of meeting Bloomington’s daily police requirements.

Idx| Time Period  | Number of Policemen Required |
---|--------------|------------------------------|
0  | 12 A.M.–6 A.M. | 12                         |
1  | 6 A.M.–12 P.M. | 8                          |
2  | 12 P.M.–6 P.M. | 6                          |
3  | 6 P.M.–12 A.M. | 15                         |


\begin{aligned}
\min            &&  48(x_0 + x_1 + x_2 + x_3) + 84(y_0 + y_1 + y_2 + y_3)\\
\text{s.t.}     &&  x_0 + x_3 + y_0 + y_3 + y_2 &&\ge 12\\
                &&  x_1 + x_0 + y_1 + y_0 + y_3 &&\ge 8\\
                &&  x_2 + x_1 + y_2 + y_1 + y_0 &&\ge 6\\
                &&  x_3 + x_2 + y_3 + y_2 + y_1 &&\ge 15
\end{aligned}

4. Each hour from 10 A.M. to 7 P.M., Bank One receives checks and must process them. Its goal is to process all the checks the same day they are received. The bank has 13 check-processing machines, each of which can process up to 500 checks per hour. It takes one worker to operate each machine. Bank One hires both full-time and part-time workers. Full-time workers work 10 A.M.–6 P.M., 11 A.M.–7 P.M., or Noon–8 P.M. and are paid $160 per day. Part-time workers work either 2 P.M.–7 P.M. or 3 P.M.–8 P.M. and are paid $75 per day. The number of checks received each hour is given below. In the interest of maintaining continuity, Bank One believes it must have at least three full-time workers under contract. Develop a cost-minimizing work schedule that processes all checks by 8 P.M.

| Time    | Checks Received | 
|---------|-----------------|
| 10 A.M. | 5,000           | 
| 11 A.M. | 4,000           | 
| Noon    | 3,000           |
| 1 P.M.  | 4,000           | 
| 2 P.M.  | 2,500           | 
| 3 P.M.  | 3,000           | 
| 4 P.M.  | 4,000           | 
| 5 P.M.  | 4,500           |
| 6 P.M.  | 3,500           | 
| 7 P.M.  | 3,000           | 

\begin{aligned}
                \quad   &   \text{var}                      & C_{\text{received}} \quad & C_0 \quad & C_{\text{processed}} \quad & C_{1}\\
10\text{ A.M.}  \quad   &   f_1                             & 5000 \quad & 5000 \quad & 500f_1 \quad & 5000 - 500f_1\\
11\text{ A.M.}  \quad   &   f_1 + f_2                       & 4000 \quad & 9000 - 500f_1 \quad & 500(f_1 + f_2) \quad & 9000 - 1000f_1 - 500f_2  \\
12\text{ P.M.}  \quad   &   f_1 + f_2 + f_3                 & 3000 \quad & 12000 - 1000f_1 - 500f_2 \quad & 500(f_1 + f_2 + f_3) \quad & ...\\
1\text{ P.M.}   \quad   &   f_1 + f_2 + f_3                 & 4000 \quad & ... \quad & 500(f_1 + f_2 + f_3) \quad & ...\\
2\text{ P.M.}   \quad   &   f_1 + f_2 + f_3 + p_1           & 2500 \quad & ... \quad & 500(f_1 + f_2 + f_3 + p_1) \quad & ...\\
3\text{ P.M.}   \quad   &   f_1 + f_2 + f_3 + p_1 + p_2     & 3000 \quad & ... \quad & 500(f_1 + f_2 + f_3 + p_1 + p_2) \quad & ...\\
4\text{ P.M.}   \quad   &   f_1 + f_2 + f_3 + p_1 + p_2     & 4000 \quad & ... \quad & 500(f_1 + f_2 + f_3 + p_1 + p_2) \quad & ...\\
5\text{ P.M.}   \quad   &   f_1 + f_2 + f_3 + p_1 + p_2     & 4500 \quad & ... \quad & 500(f_1 + f_2 + f_3 + p_1 + p_2) \quad & ...\\
6\text{ P.M.}   \quad   &   f_2 + f_3 + p_1 + p_2           & 3500 \quad & ... \quad & 500(f_2 + f_3 + p_1 + p_2) \quad & ...\\
7\text{ P.M.}   \quad   &   f_3 + p_2                       & 3000 \quad & ... \quad & 500(f_3 + p_2) \quad & ...\\[10pt]                                                                                                                                       
\end{aligned}


\begin{aligned}
&& 36500 - 8(500f_1) - 8(500f_2) - 8(500f_3) - 5(500p_1) - 5(500p_2) \quad & = 0\\
\rightarrow \quad && 36500 - 500(8f_1 + 8f_2 + 8f_3 + 5p_1 + 5p_2) \quad & = 0\\
\rightarrow \quad && 73 - (8f_1 + 8f_2 + 8f_3 + 5p_1 + 5p_2) \quad & = 0\\
\rightarrow \quad && 8f_1 + 8f_2 + 8f_3 + 5p_1 + 5p_2 \quad & = 73\\
\end{aligned}


\begin{aligned}
\min \quad & 160(f_1 + f_2 + f_3) + 75(p_1 + p_2)\\
\text{s.t.} \quad   &   8f_1 + 8f_2 + 8f_3 + 5p_1 + 5p_2 &= 73\\
            \quad   &   f_1 + f_2 + f_3 &\ge 3\\
            \quad   &   f_1 + f_2 + f_3 + p_1 + p_2 &\le 13\\
            \quad   &   f_1, f_2, f_3, p_1, p_2 &\ge 0
\end{aligned}

In [11]:
f_i = [1, 2, 3]
p_i = [1, 2]

f = LpVariable.dicts("f", f_i, lowBound=0, cat="Integer")
p = LpVariable.dicts('p', p_i, lowBound=0, cat="Integer")

model = LpProblem("min_cost", LpMinimize)

model += lpSum([160 * f[i] for i in f_i] + [75 * p[j] for j in p_i])

model += lpSum([8 * f[i] for i in f_i] + [5 * p[j] for j in p_i]) == 73

model += lpSum([f[i] for i in f_i]) >= 3

model += lpSum([f[i] for i in f_i] + [p[j] for j in p_i]) <= 13

model.solve()

for k, v in f.items():
    print(f"f{k} = {v.varValue}")
print()
for k, v in p.items():
    print(f"p{k} = {v.varValue}")
print()
print(f"Cost: ${model.objective.value():,.2f}")

f1 = 6.0
f2 = 0.0
f3 = 0.0

p1 = 5.0
p2 = 0.0

Cost: $1,335.00


In [12]:
const_1 = np.sum([8 * f[i].varValue for i in f_i] + [5 * p[j].varValue for j in p_i])
const_2 = np.sum([f[i].varValue for i in f_i])
const_3 = np.sum([f[i].varValue for i in f_i] + [p[j].varValue for j in p_i])

const_1 == 73, const_2 >= 3, const_3 <= 13, [f[i].varValue >= 0 for i in f_i], [p[i].varValue >= 0 for i in p_i]

(True, True, True, [True, True, True], [True, True])